In [109]:
import pandas as pd
import numpy as np
import geopandas as gpd
from bs4 import BeautifulSoup
import h3
from shapely.geometry import Polygon

# Data Input

In [110]:
condo_df = pd.read_parquet('../data/L1/housing_condo_transaction.parquet')
ec_df = pd.read_parquet('../data/L1/housing_ec_transaction.parquet')
hdb_df = pd.read_parquet('../data/L1/housing_hdb_transaction.parquet')
unique_df = pd.read_parquet('../data/L1/housing_unique_searched.parquet')
amenity_df = pd.read_parquet('../data/L1/amenity.parqeut')

# filter for only first result from onemap then
unique_df = unique_df.loc[unique_df['search_result']==0]
unique_df = unique_df.rename({'LATITUDE':'lat','LONGITUDE':'lon'}, axis=1)
unique_df['lat'] = unique_df['lat'].astype(float)
unique_df['lon'] = unique_df['lon'].astype(float)

# unique name for ec, condo and hdb
condo_df['unique_name'] = condo_df['Project Name'] + ' ' + condo_df['Street Name']
ec_df['unique_name'] = ec_df['Project Name'] + ' ' + ec_df['Street Name']
hdb_df['unique_name'] = hdb_df['block'] + ' ' + hdb_df['street_name']

In [111]:
#geojson
park_df = gpd.read_file("../data/L1/park.geojson")
park_connecter_df = gpd.read_file("../data/L1/park_connector.geojson")
waterbody_df = gpd.read_file("../data/L1/waterbody.geojson")

# Casting

In [112]:
ameneties_h12grid = [h3.latlng_to_cell(x,y, 13)  for x,y in zip(amenity_df['lat'], amenity_df['lon'])]

In [113]:
amenity_df['type'].value_counts()

type
preschool           2290
kindergarden         896
supermarket          526
school               336
gym                  159
hawker               125
water_activities      32
Name: count, dtype: int64

In [55]:
# unique_df
unique_df_h13grid = [h3.latlng_to_cell(x,y, 13)  for x,y in zip(unique_df['lat'], unique_df['lon'])]
grid_disk_selected = [h3.grid_disk(i, 5) for i in unique_df_h13grid]
polygon_crafted = [h3.cells_to_geo(i) for i in grid_disk_selected]
polygon = [Polygon(i['coordinates'][0]) for i in polygon_crafted]

In [105]:
unique_gdf = gpd.GeoDataFrame(unique_df, geometry=polygon)
unique_gdf

,search_result,SEARCHVAL,BLK_NO,ROAD_NAME,BUILDING,ADDRESS,POSTAL,X,Y,lat,lon,NameAddress,geometry
0,0,AFFINITY AT SERANGOON,26,SERANGOON NORTH AVENUE 1,AFFINITY AT SERANGOON,26 SERANGOON NORTH AVENUE 1 AFFINITY AT SERANG...,554340,32491.1666975729,38691.5738769248,1.366187,103.873675,AFFINITY AT SERANGOON SERANGOON NORTH AVENUE 1,"POLYGON ((103.87394 1.36601, 103.87393 1.36604..."
0,0,THE FLORENCE RESIDENCES,83,HOUGANG AVENUE 2,THE FLORENCE RESIDENCES,83 HOUGANG AVENUE 2 THE FLORENCE RESIDENCES SI...,538860,33974.2496525534,38761.4218198191,1.366818,103.887001,THE FLORENCE RESIDENCES HOUGANG AVENUE 2,"POLYGON ((103.88731 1.36662, 103.8873 1.36666,..."
0,0,THE GARDEN RESIDENCES,5,SERANGOON NORTH VIEW,THE GARDEN RESIDENCES,5 SERANGOON NORTH VIEW THE GARDEN RESIDENCES S...,554345,32143.3130448852,38637.3503580317,1.365697,103.870549,THE GARDEN RESIDENCES SERANGOON NORTH VIEW,"POLYGON ((103.87085 1.36551, 103.87084 1.36555..."
0,0,THE PROMENADE@PELIKAT,183,JALAN PELIKAT,THE PROMENADE@PELIKAT,183 JALAN PELIKAT THE PROMENADE@PELIKAT SINGAP...,537643,34069.5234887367,37488.7720274143,1.355309,103.887857,THE PROMENADE@PELIKAT JALAN PELIKAT,"POLYGON ((103.88814 1.35513, 103.88813 1.35516..."
0,0,137 JALAN PELIKAT SINGAPORE 537620,137,JALAN PELIKAT,NIL,137 JALAN PELIKAT SINGAPORE 537620,537620,33899.2988302857,37403.9616515934,1.354542,103.886328,PRIMO RESIDENCES JALAN PELIKAT,"POLYGON ((103.88663 1.35437, 103.88662 1.35441..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0,FERNVALE WOODS,457B,SENGKANG WEST ROAD,FERNVALE WOODS,457B SENGKANG WEST ROAD FERNVALE WOODS SINGAPO...,792457,32168.7083977796,41592.4737811014,1.392422,103.870778,457B SENGKANG WEST RD,"POLYGON ((103.87107 1.3922, 103.87106 1.39224,..."
0,0,ALKAFF VISTA,106B,BIDADARI PARK DRIVE,ALKAFF VISTA,106B BIDADARI PARK DRIVE ALKAFF VISTA SINGAPOR...,342106,32211.8843447613,35092.917645948,1.333642,103.871165,106B BIDADARI PK DR,"POLYGON ((103.87143 1.33345, 103.87142 1.33348..."
0,0,FERNVALE WOODS,458B,SENGKANG WEST ROAD,FERNVALE WOODS,458B SENGKANG WEST ROAD FERNVALE WOODS SINGAPO...,792458,32104.0491439952,41649.5559899863,1.392938,103.870197,458B SENGKANG WEST RD,"POLYGON ((103.87049 1.39272, 103.87048 1.39276..."
0,0,ALKAFF VISTA,105A,BIDADARI PARK DRIVE,ALKAFF VISTA,105A BIDADARI PARK DRIVE ALKAFF VISTA SINGAPOR...,341105,32130.1762556639,35133.2631429676,1.334007,103.870431,105A BIDADARI PK DR,"POLYGON ((103.87074 1.33381, 103.87073 1.33385..."


In [101]:
amenity_gdf = gpd.GeoDataFrame(
    amenity_df, 
    geometry=gpd.points_from_xy(amenity_df.lon, amenity_df.lat),
    crs="EPSG:4326"  # Coordinate Reference System
)

In [103]:
# amenity_gdf

In [104]:
# sales tranfasction
private_df = pd.concat([condo_df,ec_df])
private_df.columns

Index(['Project Name', 'Transacted Price ($)', 'Area (SQFT)',
       'Unit Price ($ PSF)', 'Sale Date', 'Street Name', 'Type of Sale',
       'Type of Area', 'Area (SQM)', 'Unit Price ($ PSM)', 'Nett Price($)',
       'Property Type', 'Number of Units', 'Tenure', 'Postal District',
       'Market Segment', 'Floor Level', 'unique_name'],
      dtype='object')

In [16]:
condo_df.columns

Index(['Project Name', 'Transacted Price ($)', 'Area (SQFT)',
       'Unit Price ($ PSF)', 'Sale Date', 'Street Name', 'Type of Sale',
       'Type of Area', 'Area (SQM)', 'Unit Price ($ PSM)', 'Nett Price($)',
       'Property Type', 'Number of Units', 'Tenure', 'Postal District',
       'Market Segment', 'Floor Level'],
      dtype='object')